In [1]:
from datasets import load_dataset

dataset = load_dataset("glue", "cola")
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["sentence"])

train_dataset = dataset["train"].map(tokenize_dataset)
validation_dataset = dataset["validation"].map(tokenize_dataset)
test_dataset = dataset["test"].map(tokenize_dataset)

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [4]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tf_train_dataset = model.prepare_tf_dataset(train_dataset, batch_size=16, shuffle=True, tokenizer=tokenizer)
tf_validation_dataset = model.prepare_tf_dataset(validation_dataset, batch_size=16, shuffle=True, tokenizer=tokenizer)
tf_test_dataset = model.prepare_tf_dataset(test_dataset, batch_size=16, shuffle=True, tokenizer=tokenizer)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [6]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(3e-5))  # No need for loss argument!

model.fit(tf_train_dataset,
          epochs = 3)

Epoch 1/3
534/534 [==============================] - 508s 924ms/step - loss: 0.5273
Epoch 2/3
534/534 [==============================] - 500s 938ms/step - loss: 0.3194
Epoch 3/3
534/534 [==============================] - 571s 1s/step - loss: 0.1717


In [ ]:
# Done.